In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 27.7 MB/s 
     |████████████████████████████████| 209 kB 69.9 MB/s 
     |████████████████████████████████| 81 kB 9.5 MB/s 
     |████████████████████████████████| 78 kB 8.0 MB/s 
     |████████████████████████████████| 112 kB 76.5 MB/s 
     |████████████████████████████████| 147 kB 66.7 MB/s 
     |████████████████████████████████| 49 kB 7.0 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=130352d3f38829f21f7f6ae3f5b16a77ba25eb26606ca39b847766ff67063c2f
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
import pandas as pd
import numpy as np
from sklearn import datasets, ensemble
import optuna
from lightgbm import LGBMRegressor
from tqdm import tqdm

from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor

from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.decomposition import PCA, KernelPCA

In [ ]:
path ='/content/drive/MyDrive/Datasets/lg_aimers/lg aimers/'
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
sample_submission = pd.read_csv(path + 'sample_submission.csv')
y_feature_spec_info = pd.read_csv(path + 'meta/y_feature_spec_info.csv')

In [ ]:
train.columns

Index(['ID', 'X_01', 'X_02', 'X_03', 'X_04', 'X_05', 'X_06', 'X_07', 'X_08',
       'X_09', 'X_10', 'X_11', 'X_12', 'X_13', 'X_14', 'X_15', 'X_16', 'X_17',
       'X_18', 'X_19', 'X_20', 'X_21', 'X_22', 'X_23', 'X_24', 'X_25', 'X_26',
       'X_27', 'X_28', 'X_29', 'X_30', 'X_31', 'X_32', 'X_33', 'X_34', 'X_35',
       'X_36', 'X_37', 'X_38', 'X_39', 'X_40', 'X_41', 'X_42', 'X_43', 'X_44',
       'X_45', 'X_46', 'X_47', 'X_48', 'X_49', 'X_50', 'X_51', 'X_52', 'X_53',
       'X_54', 'X_55', 'X_56', 'Y_01', 'Y_02', 'Y_03', 'Y_04', 'Y_05', 'Y_06',
       'Y_07', 'Y_08', 'Y_09', 'Y_10', 'Y_11', 'Y_12', 'Y_13', 'Y_14'],
      dtype='object')

In [ ]:
train.shape

(39607, 71)

In [ ]:
X = train[['X_01', 'X_02', 'X_03', 'X_05', 'X_06', 'X_07', 'X_08',
       'X_09', 'X_12', 'X_14', 'X_15', 'X_16','X_17',
       'X_18', 'X_19', 'X_20', 'X_21', 'X_22', 'X_24', 'X_25', 'X_26',
       'X_27', 'X_28', 'X_29', 'X_30', 'X_31', 'X_32', 'X_33', 'X_35',
       'X_37', 'X_38', 'X_39', 'X_40', 'X_41', 'X_42', 'X_43', 'X_44',
       'X_46', 'X_47', 'X_49', 'X_50', 'X_51', 'X_52', 'X_53',
       'X_54', 'X_55', 'X_56']]
y = train[['Y_01', 'Y_02', 'Y_03', 'Y_04', 'Y_05', 'Y_06', 'Y_07',
       'Y_08', 'Y_09', 'Y_10', 'Y_11', 'Y_12', 'Y_13', 'Y_14']]

In [ ]:
#test columns
target = test[X.columns] 

In [ ]:
#evaluation metrics
def lg_nrmse(gt, preds):
    all_nrmse = []
    for idx in range(14): 
        rmse = mean_squared_error(gt.iloc[:,idx], preds[:,idx], squared = False)
        nrmse = rmse / np.mean(np.abs(gt.iloc[:,idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:15])
    return score

In [ ]:
def objective_cat(trial):
    """
    Objective function to tune a `CatBoostRegressor` model.
    """
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


    params = {
        'objective': 'regression',
        'verbose': -1,
        'metric': 'mse', 
        'num_leaves': trial.suggest_int('num_leaves', 2, 1024, step=1, log=True), 
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.7, 1.0),
        'reg_alpha': trial.suggest_uniform('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_uniform('reg_lambda', 0.0, 10.0),
        'max_depth': trial.suggest_int('max_depth',3, 15),
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-8, 1e-2),
        'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_loguniform('subsample', 0.4, 1),
        'random_state':42
    }


    model = MultiOutputRegressor(LGBMRegressor(**params))
    model.fit(x_train, y_train)

    pred = model.predict(x_test)

    rmse = lg_nrmse(y_test, pred)

    return rmse


In [ ]:
study = optuna.create_study(direction="minimize",sampler=sampler)
study.optimize(objective_cat, n_trials=10)

[I 2022-08-17 11:39:28,448] A new study created in memory with name: no-name-345f150c-7aab-465b-845e-41df97744c5c
[I 2022-08-17 11:40:02,907] Trial 0 finished with value: 2.0013529301992703 and parameters: {'num_leaves': 17, 'colsample_bytree': 0.9852142919229748, 'reg_alpha': 0.7319939418114051, 'reg_lambda': 5.986584841970366, 'max_depth': 5, 'learning_rate': 8.62913219007185e-08, 'n_estimators': 268, 'min_child_samples': 88, 'subsample': 0.6938533737439828}. Best is trial 0 with value: 2.0013529301992703.
[I 2022-08-17 11:41:09,413] Trial 1 finished with value: 2.001349360852823 and parameters: {'num_leaves': 152, 'colsample_bytree': 0.7061753482887407, 'reg_alpha': 0.9699098521619943, 'reg_lambda': 8.324426408004218, 'max_depth': 5, 'learning_rate': 1.2329623163659816e-07, 'n_estimators': 632, 'min_child_samples': 34, 'subsample': 0.6469661675743767}. Best is trial 1 with value: 2.001349360852823.
[I 2022-08-17 11:44:06,086] Trial 2 finished with value: 2.0011961583562776 and param

In [ ]:
params['tree_method'] = 'gpu_hist'
params['metric'] = 'mse'
params['objective'] = 'regression'

In [ ]:
#best parameters
params = {'num_leaves': 667,
        'colsample_bytree': 0.709580279206433,'reg_alpha': 0.9043953301281176, 'reg_lambda': 9.401181547510408, 
          'max_depth': 15, 'learning_rate': 0.006737654559424242, 'n_estimators': 702, 'min_child_samples': 6, 'subsample': 0.6726174877527384, 'random_state': 97}
params

{'num_leaves': 667,
 'colsample_bytree': 0.709580279206433,
 'reg_alpha': 0.9043953301281176,
 'reg_lambda': 9.401181547510408,
 'max_depth': 15,
 'learning_rate': 0.006737654559424242,
 'n_estimators': 702,
 'min_child_samples': 6,
 'subsample': 0.6726174877527384,
 'random_state': 97}

In [ ]:
#drop 10 11 13 34 36 45 and meaningless binary 4 features
NRMSES = []
xgb_pred = np.zeros((target.shape[0], 14))

kf = KFold(n_splits = 5, random_state = 42, shuffle = True)
for i, idx in enumerate(kf.split(X, y)) :
    tr_x, tr_y = pd.DataFrame(X).loc[idx[0]], y.loc[idx[0]]
    val_x, val_y = pd.DataFrame(X).loc[idx[1]], y.loc[idx[1]]
    
    # cb = CatBoostRegressor(random_state = 42, max_depth = 5, n_estimators = 10000, learning_rate = 0.03, objective = 'MultiRMSE')
    xgb_model = MultiOutputRegressor(LGBMRegressor(**params))
    xgb_model.fit(tr_x, tr_y)
    
    val_pred = xgb_model.predict(val_x)
    NRMSE = lg_nrmse(val_y, val_pred)
    print(f"{i + 1} Fold NRMSE = {NRMSE}")
    NRMSES.append(NRMSE)
    
    fold_pred = xgb_model.predict(target) / kf.n_splits
    xgb_pred += fold_pred

1 Fold NRMSE = 1.9265646020938139
2 Fold NRMSE = 1.962299715290762
3 Fold NRMSE = 1.9378280386925806
4 Fold NRMSE = 1.9417332802381673
5 Fold NRMSE = 1.947418036278224


In [ ]:
fold_pred.shape

(39608, 14)

In [ ]:
np.mean(NRMSES) #drop 10 11 13 34 36 45

1.9431687345187096

KernelPCA - 수행 연산이 커서 램 초과   
PCA 46 features 10 trial - 1.97673  
PCA 20 features 10 trial - 1.97775  
PCA 2 features 10 trial - 1.98553  
변수 6개 제거 10 trial - Bad  
변수 4개 제거 10 trial - 1.9545235045556772

1. 다음은 Feature creation & Transformation에 집중

2. pearson correlation과 Recursive Feature elimination으로 feature selection 결과  
 drop된 변수가 다른 타겟에서는 유효한 feature일 수 있어 해당 과정은 skip

In [ ]:
submission = pd.read_csv(path + 'sample_submission.csv')

In [ ]:
submission.iloc[:, 1:] = xgb_pred

In [ ]:
submission

,ID,Y_01,Y_02,Y_03,Y_04,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
0,TEST_00001,1.459748,1.257368,1.148312,14.369735,31.498536,16.762434,3.118767,-26.025294,-26.085722,-22.183285,24.585884,-25.979340,-25.952320,-25.985551
1,TEST_00002,1.487789,1.242707,1.179512,13.225185,31.117919,16.584905,3.166754,-26.177322,-26.146346,-22.295515,24.318602,-26.076413,-26.086933,-26.082135
2,TEST_00003,1.415935,1.113620,1.078659,14.157764,31.966917,16.799821,3.124430,-26.017910,-26.026545,-22.120712,24.537324,-25.949710,-25.974065,-25.997588
3,TEST_00004,1.438326,1.171882,1.064473,14.815712,32.336348,17.092729,3.061496,-25.685576,-25.698289,-21.805747,24.813111,-25.639193,-25.604159,-25.667916
4,TEST_00005,1.373507,1.031568,0.983528,15.109689,31.728957,16.965819,3.118538,-25.728455,-25.757977,-21.865977,24.733149,-25.602172,-25.690905,-25.700489
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39603,TEST_39604,1.263564,0.974986,1.010338,12.607240,30.724694,16.674187,3.193534,-26.489506,-26.460049,-22.878270,24.404599,-26.339227,-26.350053,-26.399294
39604,TEST_39605,1.233073,0.846554,0.891262,13.920025,31.326927,16.776048,3.203536,-26.363836,-26.374498,-22.847352,24.447266,-26.267706,-26.319201,-26.321692
39605,TEST_39606,1.225532,0.921590,0.943529,13.171593,31.012374,16.591049,3.164640,-26.532860,-26.557860,-22.861587,24.264781,-26.515167,-26.478997,-26.514314
39606,TEST_39607,1.173703,0.838975,0.891508,13.942257,30.906782,16.799764,3.170209,-26.461122,-26.459778,-22.920804,24.479885,-26.413808,-26.429075,-26.435784


In [ ]:
submission.to_csv(path + 'lgbm_kfold_optuna_final_best_parameter.csv', index = False)